### 1. Import thư viện

In [44]:
import requests
import json
from time import sleep

### 2. Khai báo tên file

In [45]:
full_detail_match_file_name = r'D:\Learning\ML\BTL\get_raw_data\raw_data\detail_match_full.json'
full_matchID_file_name = r'D:\Learning\ML\BTL\get_raw_data\raw_data\matchID_full.json'
remain_matchIDs_file_name = r'D:\Learning\ML\BTL\get_raw_data\raw_data\remain_matchIDs.json'

### 3. Khai báo Header và API key cho Request

In [46]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0",
    "Accept-Language": "vi,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}

In [47]:
api_key = 'RGAPI-6ddfe143-be5c-4857-ab01-fb17c0280e97'

### 4. Thông tin chi tiết trận đấu

In [48]:
with open(full_matchID_file_name, 'r', encoding='utf-8') as file:
    matchIDs = json.load(file)

In [49]:
detail_matches = []

In [50]:
import requests

def get_detail_match(api_key, headers, matchIDs):
    base_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/'
    detail_matches = []
    remain_matchIDs = []

    # Danh sách queueId của các trận đấu xếp hạng (Ranked)
    ranked_queue_ids = 420  # 420: Ranked Solo/Duo

    for matchID in matchIDs[:20000]:
        while True:
            url = base_url + matchID + '?api_key=' + api_key
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                
                # Lọc trận đấu có gameMode = 'CLASSIC' và queueId thuộc Ranked
                game_mode = data.get("info", {}).get("gameMode")
                queue_id = data.get("info", {}).get("queueId")
                
                if game_mode == "CLASSIC" and queue_id == ranked_queue_ids and data.get("info", {}).get("gameDuration", 0) > 1200:
                    match_detail = {}
                    
                    # Lấy thông tin các picks (championId của mỗi participant)
                    participants = data["info"].get("participants", [])
                    team_100_picks = [p["championId"] for p in participants if p["teamId"] == 100]
                    team_200_picks = [p["championId"] for p in participants if p["teamId"] == 200]
                    
                    # Lấy thông tin các bans (từ teams)
                    teams = data["info"].get("teams", [])
                    team_100_bans = [b["championId"] for b in teams[0].get("bans", [])]
                    team_200_bans = [b["championId"] for b in teams[1].get("bans", [])]
                    
                    # Kết quả trận đấu: 1 nếu team xanh thắng, -1 nếu team đỏ thắng
                    match_result = 1 if teams[0].get("win", False) else -1
                    
                    # Gán các thông tin vào match_detail
                    match_detail["team_100_picks"] = team_100_picks
                    match_detail["team_200_picks"] = team_200_picks
                    match_detail["team_100_bans"] = team_100_bans
                    match_detail["team_200_bans"] = team_200_bans
                    match_detail["result"] = match_result
                    
                    # Thêm vào danh sách các trận đấu đã lọc
                    detail_matches.append(match_detail)
                    remain_matchIDs.append(matchID)
                
                # Loại bỏ matchID đã xử lý
                matchIDs.remove(matchID)
                break
            elif response.status_code == 429:
                print(len(detail_matches))
                sleep(20)
            else:
                print('Wrong')
                break
    
    return (remain_matchIDs, detail_matches)


In [51]:
remain_matchIDs, detail_matches = get_detail_match(api_key, headers, matchIDs)

50
50
98
98
142
142
142
184
184
225
225
270
270
318
318
356
356
415
415
415
461
461
517
517
577
577
632
632
687
687
734
734
789
789
833
883
883
929
970
970
1026
1026
1026
1062
1062
1062
1117
1117
1117
1173
1224
1277
1277
1332
1332
1376
1376
1376
1424
1474
1526
1526
1558
1558
1558
1607
1607
1607
1662
1662
1708
1708
1759
1759
1818
1818
1867
1867
1920
1920
1970
2018
2018
2068
2068
2108
2108
Wrong
2160
2160
2160
2211
2211
2211
2257
2257
2257
2312
2312
2312
2368
2368
2368
2413
2413
2413
2463
2463
2463
2515
2515
2515
2566
2566
2566
2621
2621
2621
2680
2680
2680
2725
2725
2773
2818
2818
2818
2870
2870
2924
2924
2973
2973
3023
3023
3081
3081
3132
3188
3188
3239
3290
3290
3341
3341
3402
3402
3456
3456
3500
3500
3500
3547
3547
3547
3601
3601
3601
3653
3653
3653
3709
3755
3810
3854
3854
3912
3912
3959
3959
4015
4015
4060
4060
4114
4114
4163
4163
4216
4216
4274
4274
4330
4330
4380
4380
4427
4427
4475
4475
4525
4525
4573
4573
4631
4631
4688
4688
4688
4740
4740
4740
4789
4789
4841
4841
4884
4884
493

In [52]:
# Ghi dữ liệu
with open(full_detail_match_file_name, 'w', encoding='utf-8') as file:
    json.dump(detail_matches, file, ensure_ascii=False, indent=4)

In [53]:
# Ghi dữ liệu
with open(remain_matchIDs_file_name, 'w', encoding='utf-8') as file:
    json.dump(remain_matchIDs, file, ensure_ascii=False, indent=4)